## 데이터 변환 및 병합

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import chardet
import joblib
import numpy as np
import pandas as pd

### 작업 경로 확인 및 변경

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
sorted(os.listdir())

### 실습 데이터셋 준비

In [ ]:
# z 파일을 읽고 price, df1, df2에 할당합니다.
price, df1, df2 = joblib.load(filename = 'APT_Price_Prep.z')

In [ ]:
# price의 정보를 확인합니다.
price.info()

In [ ]:
# price의 처음 5행을 출력합니다.
price.head()

### Long type 데이터프레임 생성

In [ ]:
# 두 범주형 변수의 빈도수를 elong에 할당합니다.
# [참고] elong은 구와 금액구간이 멀티 인덱스(행이름)인 시리즈입니다.
elong = price[['구', '금액구간']].value_counts().sort_index()
elong.head()

In [ ]:
# elong의 행이름을 초기화하고 기존 행이름을 열로 추가합니다.
# [참고] 마지막 열이름은 'count'로 자동 생성되었습니다.
elong = elong.reset_index()
elong.head()

In [ ]:
# elong의 마지막 열이름을 변경합니다.
elong = elong.rename(columns = {'count': '매매건수'})

### Long type을 Wide type으로 변환

In [ ]:
# 범주형 변수로 행(인덱스)과 열(컬럼)을 설정하고 연속형 변수로 채웁니다.
widen = elong.pivot(index = '구', 
                    columns = '금액구간', 
                    values = '매매건수')
widen.head()

In [ ]:
# widen의 행이름을 출력합니다.
# [참고] 행이름에 name(이름) 속성이 있고 값은 '구'입니다.
widen.index

In [ ]:
# 행이름의 name을 출력합니다.
# [참고] widen의 행이름을 초기화하면 인덱스 name을 열이름으로 자동 적용합니다.
widen.index.name

### widen 행이름 초기화

In [ ]:
# widen의 행이름을 초기화하고 기존 행이름을 열로 추가합니다.
# [참고] 행이름의 name 값인 '구'를 열이름으로 자동 적용했습니다.
widen = widen.reset_index()
widen.head()

In [ ]:
# widen의 열이름을 출력합니다.
# [참고] 열이름에 name(이름) 속성이 있고 값은 '금액구간'입니다.
widen.columns

In [ ]:
# 열이름의 name을 출력합니다.
widen.columns.name

In [ ]:
# 열이름의 name에 None을 할당하여 삭제합니다.
widen.columns.name = None
widen.head()

### Wide type을 Long type으로 변환

In [ ]:
# 두 개 이상의 열이름을 지정하여 데이터프레임을 세로로 늘입니다.
elong = widen.melt(id_vars = '구', 
                   value_vars = ['5천 미만', '5천 이상'], 
                   var_name = '구분', 
                   value_name = '건수')
elong.head()

In [ ]:
# elong을 구로 오름차순 정렬하고 행이름을 초기화한 다음 처음 5행을 출력합니다.
elong.sort_values(by = ['구']).reset_index(drop = True).head()

### 데이터프레임 결합

In [ ]:
# 두 데이터프레임의 열이름이 순서까지 정확하게 같은지 확인합니다.
# [참고] equals() 함수는 원소의 순서만 달라도 False를 반환합니다.
df1.columns.equals(other = df2.columns)

In [ ]:
# 열이름이 같은 두 데이터프레임을 행(세로) 방향으로 결합한 결과를 출력합니다.
# [참고] 두 데이터프레임의 기존 행이름을 유지합니다.
pd.concat(objs = [df1, df2])

In [ ]:
# 두 데이터프레임을 세로로 결합하고 행이름을 초기화합니다.
pd.concat(objs = [df1, df2], ignore_index = True)

In [ ]:
# df2의 일부 열이름을 변경합니다.
df2 = df2.rename(columns = {'아파트': '아파트명'})

In [ ]:
# 열이름이 다른 두 데이터프레임을 행(세로) 방향으로 결합한 결과를 출력합니다.
pd.concat(objs = [df1, df2], ignore_index = True)

### 병합 데이터셋 준비

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 출력합니다.
sorted(os.listdir())

In [ ]:
# 상세정보 xlsx 파일을 읽고 데이터프레임을 생성합니다.
detail = pd.read_excel(io = 'APT_Detail_Seoul_2022.xlsx')

In [ ]:
# detail의 정보를 확인합니다.
detail.info()

In [ ]:
# detail의 처음 5행을 출력합니다.
detail.head()

### 외래키 확인 및 전처리

In [ ]:
# price 아파트와 detail 아파트명에서 일치하는 원소 개수를 확인합니다.
len(set(price['아파트']) & set(detail['아파트명']))

In [ ]:
# price 아파트에 있지만 detail 아파트명에 없는 원소 개수를 확인합니다.
len(set(price['아파트']) - set(detail['아파트명']))

In [ ]:
# price 주소와 detail 지번주소에서 일치하는 원소 개수를 확인합니다.
len(set(price['주소']) & set(detail['지번주소']))

In [ ]:
# price의 처음 5행을 출력합니다.
# [참고] price 주소가 '서울특별시'로 시작합니다.
price['주소'].head()

In [ ]:
# detail의 처음 5행을 출력합니다.
# [참고] detail 지번주소는 '서울시'로 시작합니다.
detail['지번주소'].head()

In [ ]:
# price 주소에서 '특별'을 삭제합니다.
price['주소'] = price['주소'].str.replace(pat = '특별', repl = '')

In [ ]:
# 외래키에서 일치하는 원소 개수를 다시 확인합니다.
len(set(price['주소']) & set(detail['지번주소']))

In [ ]:
# price 주소에 있지만 detail 지번주소에 없는 원소 개수를 확인합니다.
len(set(price['주소']) - set(detail['지번주소']))

### [참고] 비복원추출

In [ ]:
# 반복문으로 로또 번호를 10번 출력합니다.
for i in range(10):
    np.random.seed(seed = 1)
    lotto = np.random.choice(a = range(1, 46), 
                             size = 6, 
                             replace = False)
    lotto.sort()
    print(lotto)

### [참고] 복원추출

In [ ]:
# 시드를 고정합니다.
np.random.seed(seed = 1)

In [ ]:
# 1~4의 정수에서 5개를 복원추출하여 nums에 할당합니다.
nums = np.random.choice(a = range(1, 5), size = 5)
nums

In [ ]:
# nums를 시리즈로 변환합니다.
nums = pd.Series(data = nums)
nums

### [참고] 중복 원소 확인 함수

In [ ]:
# 시리즈 원소가 중복이면 True, 아니면 False를 반환합니다.
# [참고] keep 매개변수에 'first'(기본값), 'last' 또는 False를 지정합니다.
nums.duplicated(keep = False)

In [ ]:
# nums의 중복 원소를 모두 선택합니다.
nums[nums.duplicated(keep = False)]

In [ ]:
# 시리즈의 중복 원소 중 첫 번째 원소만 False, 나머지 원소는 True를 반환합니다.
# [참고] keep 매개변수에 'last'를 지정하면 시리즈의 중복 원소 중 마지막 원소만 False, 
# 나머지 원소는 True를 반환합니다.
nums.duplicated()

In [ ]:
# nums의 중복 원소 중 첫 번째 원소를 제외한 나머지 원소를 선택합니다.
# [참고] 중복이 아닌 원소는 False이므로 선택하지 않습니다.
nums[nums.duplicated()]

In [ ]:
# nums에서 중복이 아닌 원소와 중복 원소 중 첫 번째 원소만 선택합니다.
# [참고] ~ 연산자는 진리값을 반전합니다.
nums[~nums.duplicated()]

### 데이터프레임 중복 확인 및 제거

In [ ]:
# detail에서 지번주소의 원소가 중복이면 True, 아니면 False인 dup을 생성합니다.
dup = detail['지번주소'].duplicated(keep = False)

In [ ]:
# detail에서 지번주소가 중복인 행을 선택하고 지번주소로 오름차순 정렬합니다.
detail[dup].sort_values(by = ['지번주소'])

In [ ]:
# detail에서 지번주소가 중복인 원소 중 첫 번째가 아닌 원소의 개수를 확인합니다.
detail['지번주소'].duplicated().sum()

In [ ]:
# detail에서 지번주소가 중복이 아니거나 중복인 첫 번째 행만 남깁니다.
# [참고] ~ 연산자는 진리값을 반전합니다.
detail = detail[~detail['지번주소'].duplicated()]

In [ ]:
# detail의 행 개수를 확인합니다.
# [참고] detail의 행 개수가 감소했습니다.(10290 -> 10261)
detail.shape[0]

### 데이터프레임 병합

In [ ]:
# price와 detail에서 일치하는 열이름을 확인합니다.
set(price.columns) & set(detail.columns)

In [ ]:
# price와 detail로 내부 병합을 실행합니다. 외래키는 '주소'와 '지번주소'입니다.
pd.merge(left = price, 
         right = detail, 
         how = 'inner', 
         left_on = '주소', 
         right_on = '지번주소')

In [ ]:
# detail의 외래키 이름을 '주소'로 변경합니다.
detail = detail.rename(columns = {'지번주소': '주소'})

In [ ]:
# price와 detail에서 일치하는 열이름을 확인합니다.
set(price.columns) & set(detail.columns)

In [ ]:
# 외래키 이름이 같으면 on 매개변수를 사용합니다.
# [참고] 두 데이터프레임에서 외래키 이름만 같으면 on 매개변수를 생략할 수 있습니다.
apt = pd.merge(left = price, right = detail, how = 'inner', on = '주소')

In [ ]:
# apt의 정보를 확인합니다.
apt.info()

### 외부 파일로 저장

In [ ]:
# apt에서 불필요한 일부 열을 삭제합니다.
apt = apt.drop(columns = ['주소', '아파트ID', '아파트명'])

In [ ]:
# apt를 xlsx 파일로 저장합니다.
%time apt.to_excel(excel_writer = 'APT_Merged.xlsx', index = False)

In [ ]:
# apt를 csv 파일로 저장합니다.
%time apt.to_csv(path_or_buf = 'APT_Merged.csv', index = False)

In [ ]:
# apt를 z 파일로 저장합니다.
%time joblib.dump(value = apt, filename = 'APT_Merged.z')

## End of Document